In [153]:
from omnisafe.models.actor import GaussianLearningActor
import safety_gymnasium
import torch

env = safety_gymnasium.make('SafetyPointGoal1-v0')

def create_random_agent(env, hidden_layers=[255,255,255,255], activation='relu', weight_initialization_mode='orthogonal'):
    obs_space = env.observation_space
    act_space = env.action_space
    return GaussianLearningActor(obs_space, act_space, hidden_layers, activation=activation, weight_initialization_mode=weight_initialization_mode)

In [154]:
env.obs_space_dict

Dict('accelerometer': Box(-inf, inf, (3,), float64), 'velocimeter': Box(-inf, inf, (3,), float64), 'gyro': Box(-inf, inf, (3,), float64), 'magnetometer': Box(-inf, inf, (3,), float64), 'goal_lidar': Box(0.0, 1.0, (16,), float64), 'hazards_lidar': Box(0.0, 1.0, (16,), float64), 'vases_lidar': Box(0.0, 1.0, (16,), float64))

In [155]:
env.action_space

Box(-1.0, 1.0, (2,), float64)

In [156]:
import numpy as np

def run_trajectory(env, agent, num_data_points=100, cost_window=200, deterministic=True):
    observation, info = env.reset()
    episode_over = False
    data = []
    costs = []
    # gather data
    while not episode_over:
        obs_tensor = torch.from_numpy(observation).float()
        action = agent.predict(obs_tensor, deterministic=True).detach().numpy()
        data.append(np.append(observation, action))
        observation, reward, cost, terminated, truncated, info = env.step(action)
        costs.append(cost)
        episode_over = terminated or truncated
    env.close()
    # pick num_data_points out of the data and calculate cost in the next cost_window steps
    indices = np.random.choice(np.arange(len(data)), size=100)
    chosen_data = np.array(data)[indices]
    labels = []
    for i in indices:
        if i + cost_window >= len(costs):
            labels.append(sum(costs[i:]))
        else:
            labels.append(sum(costs[i:i+cost_window]))
    return chosen_data, np.array(labels)

In [157]:
data, labels = run_trajectory(env, create_random_agent(env))
labels

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [158]:
print(data.shape)

(100, 62)


In [159]:
def generate_dataset(env, amount=1000):
    data = []
    labels = []
    for i in range(amount//100):
        data_i, labels_i = run_trajectory(env, create_random_agent(env))
        data.append(data_i)
        labels.append(labels_i)
    return np.concatenate(data, axis=0), np.concatenate(labels, axis=0)

In [160]:
data, labels = generate_dataset(env)

In [161]:
data

array([[ 1.66896804,  2.93446876,  9.81      , ...,  0.01686774,
         0.91074193,  0.76333821],
       [ 1.55027913,  1.37797954,  9.81      , ...,  0.        ,
         1.16064906,  0.47104338],
       [ 1.59597247,  1.47476916,  9.81      , ...,  0.        ,
         0.80572146,  0.74217308],
       ...,
       [-1.51496598, 11.15416165,  9.81      , ...,  0.        ,
        -1.80254471, -0.8403492 ],
       [-1.12851019, -8.42128635,  9.81      , ...,  0.        ,
        -1.30260324, -1.78134608],
       [-1.45465617,  1.62278407,  9.81      , ...,  0.09778371,
        -1.66756952, -0.52859056]])

In [162]:
data.shape

(1000, 62)

In [163]:
labels

array([32., 51.,  0., 40., 23., 51., 19., 34., 32., 55., 51., 23., 27.,
       47., 32., 34., 16., 49., 32., 51., 42., 47., 35., 51., 64., 34.,
       21., 27., 49., 22., 32., 34., 51., 28., 47., 23., 32., 51., 23.,
       51., 43., 42., 34., 42., 32.,  0., 34., 49., 49., 34., 36., 32.,
       51., 32., 23., 51., 21., 19., 32., 24.,  0., 19., 55., 45., 19.,
       18., 34., 43., 19., 34., 25., 23., 48., 51., 19., 54., 51., 50.,
       19., 23., 66., 18., 40., 30., 23.,  0.,  0., 38., 34., 37., 51.,
       23., 51., 19., 34.,  0., 19.,  0., 34., 35., 34., 27., 45., 46.,
       25., 42., 42., 13.,  0., 32., 34., 42., 12., 46., 42.,  0., 27.,
        0., 34., 19., 40., 33., 25.,  0., 34., 34., 37., 33., 34., 45.,
        0., 34.,  0., 33., 19., 37., 50., 33.,  0., 34.,  0.,  0., 46.,
       42., 34., 45., 19., 19., 23., 32., 25.,  0., 29.,  0.,  0., 26.,
       46., 46., 34.,  0., 12., 34., 19.,  0., 29., 41., 19., 20., 36.,
       33., 46., 44., 46., 25.,  0., 52., 19.,  0., 46., 34., 27

In [164]:
labels.shape

(1000,)